## Compile Static

In [1]:
%%writefile matrix_multiply.c
#include <stdio.h>

void matrix_multiply(int n, int A[][n], int B[][n], int C[][n]) {
    for (int i = 0; i < n; i++) {
        for (int j = 0; j < n; j++) {
            C[i][j] = 0;
            for (int k = 0; k < n; k++) {
                C[i][j] += A[i][k] * B[k][j];
            }
        }
    }
}

Writing matrix_multiply.c


In [2]:
%%writefile matrix_vector_multiply.c
#include <stdio.h>

void matrix_vector_multiply(int n, int A[][n], int B[], int C[]) {
    for (int i = 0; i < n; i++) {
        C[i] = 0;
        for (int j = 0; j < n; j++) {
            C[i] += A[i][j] * B[j];
        }
    }
}

Writing matrix_vector_multiply.c


In [3]:
%%writefile main.c
#include <stdio.h>
#include <stdlib.h>

void matrix_multiply(int n, int A[][n], int B[][n], int C[][n]);
void matrix_vector_multiply(int n, int A[][n], int B[], int C[]);

int main(int argc, char *argv[]) {

    int N = atoi(argv[1]);

    // Allocate memory for matrices and vector
    int (*A)[N] = malloc(sizeof(int[N][N]));
    int (*B)[N] = malloc(sizeof(int[N][N]));
    int (*C)[N] = malloc(sizeof(int[N][N]));
    int *D = malloc(sizeof(int[N]));
    int *E = malloc(sizeof(int[N]));

    // Generate random values for matrices and vector
    for (int i = 0; i < N; i++) {
        for (int j = 0; j < N; j++) {
            A[i][j] = rand() % 10; // Generate random numbers between 0 and 9
            B[i][j] = rand() % 10;
        }
        D[i] = rand() % 10;
    }

    // Perform matrix multiplication
    matrix_multiply(N, A, B, C);

    // Perform matrix-vector multiplication
    matrix_vector_multiply(N, A, D, E);

    // Free allocated memory
    free(A);
    free(B);
    free(C);
    free(D);
    free(E);

    return 0;
}

Writing main.c


In [4]:
!gcc main.c matrix_multiply.c matrix_vector_multiply.c -static -o great_program

In [5]:
%%timeit
!./great_program 512

607 ms ± 7.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Compile Opt/Debug

In [6]:
!gcc main.c matrix_multiply.c matrix_vector_multiply.c -fPIC -shared -g -o great_program_debug.so

In [7]:
!gcc main.c matrix_multiply.c matrix_vector_multiply.c -fPIC -shared -O3 -o great_program_opt.so

In [8]:
%%timeit
!./great_program_debug.so 512

108 ms ± 64.1 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
!./great_program_opt.so 512

108 ms ± 107 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## ctypes

In [10]:
import ctypes

In [11]:
great_program_py = ctypes.CDLL('./great_program_opt.so')

In [12]:
great_program_py.main.argtypes = [ctypes.c_int, ctypes.POINTER(ctypes.c_char_p)]
great_program_py.main.restype = ctypes.c_int

In [13]:
argc = 2
argv = (ctypes.c_char_p * argc)()
argv[0] = b'./great_program_opt.so'
argv[1] = b'512'

In [14]:
%%timeit
great_program_py.main(argc, argv)

171 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Cython

In [15]:
!pip install cython

In [16]:
%load_ext cython

In [17]:
%%writefile great_program_cython.pyx
cimport cython
from libc.stdlib cimport rand, RAND_MAX, srand


def matrix_multiply(double[:, :] matrix_a,
                           double[:, :] matrix_b,
                           int n):
    """
    Performs square matrix multiplication C = A * B.

    Args:
        matrix_a (double[:, :]): Square matrix A (n x n).
        matrix_b (double[:, :]): Square matrix B (n x n).
        n (int): Dimension of the matrices (n x n).

    Returns:
        double[:, :]: Square matrix C (n x n) resulting from A * B.
    """
    cdef double[:, :] matrix_c = cython.double[:, ::1](shape=(n, n)) # C array memoryview
    cdef int i, j, k

    for i in range(n):
        for j in range(n):
            matrix_c[i, j] = 0.0 # Initialize to 0
            for k in range(n):
                matrix_c[i, j] += matrix_a[i, k] * matrix_b[k, j]
    return matrix_c


def matrix_vector_multiply(double[:, :] matrix_m,
                            double[:] vector_v,
                            int n):
    """
    Performs matrix-vector multiplication v_out = M * v_in.

    Args:
        matrix_m (double[:, :]): Square matrix M (n x n).
        vector_v (double[:]): Vector v_in (n).
        n (int): Dimension of the matrix and vector.

    Returns:
        double[:]: Vector v_out (n) resulting from M * v_in.
    """
    cdef double[:] vector_out = cython.double[::1](shape=(n,)) # C array memoryview
    cdef int i, j

    for i in range(n):
        vector_out[i] = 0.0 # Initialize to 0
        for j in range(n):
            vector_out[i] += matrix_m[i, j] * vector_v[j]
    return vector_out


def main(int n):

    cdef double[:, :] matrix_a = cython.double[:, ::1](shape=(n, n))
    cdef double[:, :] matrix_b = cython.double[:, ::1](shape=(n, n))
    cdef double[:, :] matrix_m = cython.double[:, ::1](shape=(n, n))
    cdef double[:] vector_v = cython.double[::1](shape=(n,))
    cdef int i, j

    for i in range(n):
        for j in range(n):
            matrix_a[i, j] = <double>rand() / <double>RAND_MAX
            matrix_b[i, j] = <double>rand() / <double>RAND_MAX
            matrix_m[i, j] = <double>rand() / <double>RAND_MAX
        vector_v[i] = <double>rand() / <double>RAND_MAX


    matrix_multiply(matrix_a, matrix_b, n)
    matrix_vector_multiply(matrix_m, vector_v, n)

    return 0

Writing great_program_cython.pyx


In [18]:
%%writefile setup.py
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules=cythonize("great_program_cython.pyx")
)

Writing setup.py


In [19]:
!python setup.py build_ext --inplace

Compiling great_program_cython.pyx because it changed.
[1/1] Cythonizing great_program_cython.pyx
/home/edd-ign/anaconda3/lib/python3.11/site-packages/Cython/Compiler/Main.py:381: FutureWarning: Cython directive 'language_level' not set, using '3str' for now (Py3). This has changed from earlier releases! File: /home/edd-ign/projects/matrix-multiplication2/great_program_cython.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
running build_ext
building 'great_program_cython' extension
creating build
creating build/temp.linux-x86_64-cpython-311
/home/edd-ign/anaconda3/bin/x86_64-conda-linux-gnu-cc -DNDEBUG -fwrapv -O2 -Wall -fPIC -O2 -isystem /home/edd-ign/anaconda3/include -fPIC -O2 -isystem /home/edd-ign/anaconda3/include -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -ffunction-sections -pipe -isystem /home/edd-ign/anaconda3/include -DNDEBUG -D_FORTIFY_SOURCE=2 -O2 -isystem /home/edd-ign/anaconda3/include -fPIC -I/home/edd-ign/anaconda3

In [20]:
import great_program_cython

In [21]:
great_program_cython.main(512)

NameError: name 'cython' is not defined